In [1]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

## DataFrame 생성

In [2]:
#파일 읽어오기
import os
from pyspark.sql.types import StringType

file = os.path.join("speech_.txt")

#파일 확인 및 line 별로 읽기
try:
    _f = open(file, 'r',encoding = 'utf-8')
    _lines=_f.readlines()
    _f.close()
except:
    print("An exception occurred")
    
myDf = spark.createDataFrame(_lines,StringType())
myDf.show()

+----------------------------------+
|                             value|
+----------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|
| 국민의 안전을 위해 밤낮없이 애...|
|오늘 홍조근정훈장을 받으신 중앙...|
|          사랑하는 경찰관 여러분,
|
|여러분의 헌신적 노력으로 우리의...|
| 치안의 개선은 국민의 체감으로 ...|
| 한국을 찾는 외국 관광객들도 우...|
|   올해는 ‘경찰의 날’에 맞춰 국...|
|        자랑스러운 경찰관 여러분,
|
| 경찰헌장은 “나라와 겨레를 위하...|
|    대한민국 경찰은 1945년 광복...|
| 임시정부 초대 경무국장 백범 김...|
|    광복 이후 6·25전쟁에서도 경...|
|그러나 잘못도 없지는 않았습니다...|
| 지금 경찰은 과거를 돌아보며 국...|
| 검경 수사권 조정과 자치경찰제 ...|
|정부는 경찰의 근무여건을 개선하...|
| 정부는 누구도 법 위에 군림하지...|
| 경찰헌장은 따뜻한 경찰, 의로운...|
|행사를 준비하신 민갑룡 경찰청장...|
+----------------------------------+
only showing top 20 rows



## 2. 단어로 분리해서 출력

In [3]:
from pyspark.ml.feature import RegexTokenizer

re = RegexTokenizer(inputCol="value", outputCol="wordsReg", pattern="\\s+")
reDf = re.transform(myDf)
reDf.show()

+----------------------------------+--------------------------------+
|                             value|                        wordsReg|
+----------------------------------+--------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|   [존경하는, 국민, 여러분,, ...|
| 국민의 안전을 위해 밤낮없이 애...|  [국민의, 안전을, 위해, 밤낮...|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|
|          사랑하는 경찰관 여러분,
|     [사랑하는, 경찰관, 여러분,]|
|여러분의 헌신적 노력으로 우리의...| [여러분의, 헌신적, 노력으로,...|
| 치안의 개선은 국민의 체감으로 ...|  [치안의, 개선은, 국민의, 체...|
| 한국을 찾는 외국 관광객들도 우...|  [한국을, 찾는, 외국, 관광객...|
|   올해는 ‘경찰의 날’에 맞춰 국...|    [올해는, ‘경찰의, 날’에, ...|
|        자랑스러운 경찰관 여러분,
|   [자랑스러운, 경찰관, 여러분,]|
| 경찰헌장은 “나라와 겨레를 위하...| [경찰헌장은, “나라와, 겨레를...|
|    대한민국 경찰은 1945년 광복...|    [대한민국, 경찰은, 1945년...|
| 임시정부 초대 경무국장 백범 김...|  [임시정부, 초대, 경무국장, ...|
|    광복 이후 6·25전쟁에서도 경...|    [광복, 이후, 6·25전쟁에서...|
|그러나 잘못도 없지는 않았습니다...|  [그러나, 잘못도, 없지는, 않...|
| 지금 경찰은 과거를 돌아보며 국...|  [지금, 경찰은, 과거를, 돌아...|
| 검경 수사권 조정과 자치경찰제 ...|  [검경, 수사권, 조정과, 자치...|
|정부는 경찰의 근무여건을 개선하...| [정

**++ 정리하기**<br>
콤마, 따옴표, 마침표 등 제거

In [4]:
'여러분,'.rstrip(',')

'여러분'

In [5]:
'‘경찰의'.lstrip('‘')

'경찰의'

In [6]:
'날’입니다.'.replace("’","")

'날입니다.'

In [7]:
wordList = ['존경하는', '국민', '여러분,', '경찰관', '여러분,', '일흔네', '돌', '‘경찰의', '날’입니다.']
cleaned = list()
for w in wordList:
    cleaned.append(w.lstrip("‘").rstrip("’").rstrip(".").rstrip(",").replace("’","") )
cleaned

['존경하는', '국민', '여러분', '경찰관', '여러분', '일흔네', '돌', '경찰의', '날입니다']

숫자 제거 <br>
숫자가 하나 이상 있는 경우 정규식 패턴 \d+ 를 적용한다. 숫자로 시작하는 문자열인 경우 regex.matxh(w)가 아니면 컴마등을 정리한다

In [8]:
import re
regex = re.compile('\d+') # 백슬래시는 탈출 문자임
cleaned = list()

wordList = ["1", "123", "15만","2015년에","15.1%","74.5점","8,572명을","Seoul1","Seoul"]
for w in wordList:
    if not regex.match(w):
        cleaned.append(w)
print(cleaned)

['Seoul1', 'Seoul']


udf 함수사용 <br>
udf 함수 결과는 문자열 배열 ArrayType(StringType)로 맞춰짐

In [9]:
import re
def trim(wordList):
    regmex = re.compile('\d+')
    cleaned = list()
    for w in wordList:
        if not regmex.match(w):
            cleaned.append(w.lstrip("‘").rstrip("’").rstrip(".").rstrip(",").replace("’",""))
    return cleaned

In [10]:
from pyspark.sql import functions as f
from pyspark.sql.types import ArrayType, StringType

trimUdf = f.udf(trim,ArrayType(StringType()))

In [11]:
wordsDf = reDf.withColumn('word', trimUdf(f.col('wordsReg')))
wordsDf.show()

+----------------------------------+--------------------------------+--------------------------------+
|                             value|                        wordsReg|                            word|
+----------------------------------+--------------------------------+--------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|   [존경하는, 국민, 여러분,, ...|  [존경하는, 국민, 여러분, 경...|
| 국민의 안전을 위해 밤낮없이 애...|  [국민의, 안전을, 위해, 밤낮...|  [국민의, 안전을, 위해, 밤낮...|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|[오늘, 홍조근정훈장을, 받으신...|
|          사랑하는 경찰관 여러분,
|     [사랑하는, 경찰관, 여러분,]|      [사랑하는, 경찰관, 여러분]|
|여러분의 헌신적 노력으로 우리의...| [여러분의, 헌신적, 노력으로,...| [여러분의, 헌신적, 노력으로,...|
| 치안의 개선은 국민의 체감으로 ...|  [치안의, 개선은, 국민의, 체...|  [치안의, 개선은, 국민의, 체...|
| 한국을 찾는 외국 관광객들도 우...|  [한국을, 찾는, 외국, 관광객...|  [한국을, 찾는, 외국, 관광객...|
|   올해는 ‘경찰의 날’에 맞춰 국...|    [올해는, ‘경찰의, 날’에, ...|  [올해는, 경찰의, 날에, 맞춰...|
|        자랑스러운 경찰관 여러분,
|   [자랑스러운, 경찰관, 여러분,]|    [자랑스러운, 경찰관, 여러분]|
| 경찰헌장은 “나라와 겨레를 위하...| [경찰헌장은, “나라와, 겨레를...| [경찰헌장은, “나라와, 

## 3. 불용어 구성, 출력  - 축사 전문에서 한 단어를 스스로 구성

In [12]:
from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="wordsReg", outputCol="nostops")

stopwords=list()
_stopwords=stop.getStopWords()
for e in _stopwords:
    stopwords.append(e)

kstopwords=[u"그",u"오늘"] 
for e in kstopwords:
    stopwords.append(e)

stop.setStopWords(stopwords)

#stopwords 확인
for i in stop.getStopWords():
    print (i, end="/")

i/me/my/myself/we/our/ours/ourselves/you/your/yours/yourself/yourselves/he/him/his/himself/she/her/hers/herself/it/its/itself/they/them/their/theirs/themselves/what/which/who/whom/this/that/these/those/am/is/are/was/were/be/been/being/have/has/had/having/do/does/did/doing/a/an/the/and/but/if/or/because/as/until/while/of/at/by/for/with/about/against/between/into/through/during/before/after/above/below/to/from/up/down/in/out/on/off/over/under/again/further/then/once/here/there/when/where/why/how/all/any/both/each/few/more/most/other/some/such/no/nor/not/only/own/same/so/than/too/very/s/t/can/will/just/don/should/now/i'll/you'll/he'll/she'll/we'll/they'll/i'd/you'd/he'd/she'd/we'd/they'd/i'm/you're/he's/she's/it's/we're/they're/i've/we've/you've/they've/isn't/aren't/wasn't/weren't/haven't/hasn't/hadn't/don't/doesn't/didn't/won't/wouldn't/shan't/shouldn't/mustn't/can't/couldn't/cannot/could/here's/how's/let's/ought/that's/there's/what's/when's/where's/who's/why's/would/그/오늘/

## 4. 불용어 제거하고, 출력

In [13]:
stopDf=stop.transform(wordsDf)
stopDf.show() #세번째 줄에서 '오늘' 사라진 것 확인

+----------------------------------+--------------------------------+--------------------------------+--------------------------------+
|                             value|                        wordsReg|                            word|                         nostops|
+----------------------------------+--------------------------------+--------------------------------+--------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|   [존경하는, 국민, 여러분,, ...|  [존경하는, 국민, 여러분, 경...|   [존경하는, 국민, 여러분,, ...|
| 국민의 안전을 위해 밤낮없이 애...|  [국민의, 안전을, 위해, 밤낮...|  [국민의, 안전을, 위해, 밤낮...|  [국민의, 안전을, 위해, 밤낮...|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|[오늘, 홍조근정훈장을, 받으신...|[홍조근정훈장을, 받으신, 중앙...|
|          사랑하는 경찰관 여러분,
|     [사랑하는, 경찰관, 여러분,]|      [사랑하는, 경찰관, 여러분]|     [사랑하는, 경찰관, 여러분,]|
|여러분의 헌신적 노력으로 우리의...| [여러분의, 헌신적, 노력으로,...| [여러분의, 헌신적, 노력으로,...| [여러분의, 헌신적, 노력으로,...|
| 치안의 개선은 국민의 체감으로 ...|  [치안의, 개선은, 국민의, 체...|  [치안의, 개선은, 국민의, 체...|  [치안의, 개선은, 국민의, 체...|
| 한국을 찾는 외국 관광객들도 우...|  [한국을, 찾는, 외국, 

In [14]:
# 전체 단어 빈도
## Rdd로 변환 -> flatMap 사용하여 차원 변경
stopDf.select("nostops").rdd.take(3)

[Row(nostops=['존경하는', '국민', '여러분,', '경찰관', '여러분,', '일흔네', '돌', '‘경찰의', '날’입니다.']),
 Row(nostops=['국민의', '안전을', '위해', '밤낮없이', '애쓰시는', '전국의', '15만', '경찰관', '여러분께', '먼저', '감사를', '드립니다.', '전몰·순직', '경찰관들의', '고귀한', '희생에', '경의를', '표합니다.', '유가족', '여러분께', '위로의', '마음을', '전합니다.']),
 Row(nostops=['홍조근정훈장을', '받으신', '중앙경찰학교장', '이은정', '치안감님,', '근정포장을', '받으신', '광주남부경찰서', '김동현', '경감님을', '비롯한', '수상자', '여러분께', '각별한', '축하와', '감사를', '드립니다.', '또한', '경찰', '영웅으로', '추서되신', '차일혁,', '최중락님께', '국민의', '사랑을', '전해드립니다.'])]

In [15]:
stopDf.select("nostops").rdd.flatMap(lambda x:x).take(2)

[['존경하는', '국민', '여러분,', '경찰관', '여러분,', '일흔네', '돌', '‘경찰의', '날’입니다.'],
 ['국민의',
  '안전을',
  '위해',
  '밤낮없이',
  '애쓰시는',
  '전국의',
  '15만',
  '경찰관',
  '여러분께',
  '먼저',
  '감사를',
  '드립니다.',
  '전몰·순직',
  '경찰관들의',
  '고귀한',
  '희생에',
  '경의를',
  '표합니다.',
  '유가족',
  '여러분께',
  '위로의',
  '마음을',
  '전합니다.']]

In [16]:
stopDf.select('nostops').rdd.flatMap(lambda x:x).flatMap(lambda x:x).take(3)

['존경하는', '국민', '여러분,']

In [17]:
#빈도계산하기
stopDf.select("nostops")\
    .rdd\
    .flatMap(lambda x:x).flatMap(lambda x:x)\
    .map(lambda x: (x,1))\
    .reduceByKey(lambda x,y: x+y)\
    .map(lambda x: (x[1],x[0]))\
    .sortByKey(False).take(20)

[(8, '경찰은'),
 (7, '국민의'),
 (7, '있습니다.'),
 (6, '여러분,'),
 (5, '경찰관'),
 (4, '우리의'),
 (4, '합니다.'),
 (4, '경찰의'),
 (3, '여러분께'),
 (3, '드립니다.'),
 (3, '역대'),
 (3, '가장'),
 (3, '함께'),
 (3, '것입니다.'),
 (2, '‘경찰의'),
 (2, '안전을'),
 (2, '위해'),
 (2, '먼저'),
 (2, '감사를'),
 (2, '받으신')]

In [18]:
#빈도계산하기
stopDf.select("nostops")\
    .rdd\
    .flatMap(lambda x:x).flatMap(lambda x:x)\
    .map(lambda x: x.replace("경찰은","경찰"))\
    .map(lambda x: x.replace("경찰의","경찰"))\
    .map(lambda x: (x,1))\
    .reduceByKey(lambda x,y: x+y)\
    .map(lambda x: (x[1],x[0]))\
    .sortByKey(False).take(20)

[(14, '경찰'),
 (7, '국민의'),
 (7, '있습니다.'),
 (6, '여러분,'),
 (5, '경찰관'),
 (4, '우리의'),
 (4, '합니다.'),
 (3, '여러분께'),
 (3, '드립니다.'),
 (3, '역대'),
 (3, '가장'),
 (3, '함께'),
 (3, '것입니다.'),
 (2, '‘경찰'),
 (2, '안전을'),
 (2, '위해'),
 (2, '먼저'),
 (2, '감사를'),
 (2, '받으신'),
 (2, '비롯한')]

## 5. TF-IDF를 계산하고, 출력

In [19]:
from pyspark.ml.feature import HashingTF, IDF

hashTF = HashingTF(inputCol="nostops", outputCol="hash")
hashDf = hashTF.transform(stopDf)
#hashDf.select("nostops", "hash").show(truncate=True)

idf = IDF(inputCol="hash", outputCol="idf")
idfModel = idf.fit(hashDf)
idfDf = idfModel.transform(hashDf)

#TF-IDF 출력
for e in idfDf.select("idf").take(5):
     print(e)

Row(idf=SparseVector(262144, {162: 2.3979, 80732: 2.3979, 118171: 2.9632, 127225: 2.3979, 142775: 1.9924, 160086: 1.2993, 172380: 2.3979, 254275: 2.3979}))
Row(idf=SparseVector(262144, {5341: 2.3979, 6304: 2.3979, 30732: 2.3979, 39431: 1.7047, 43098: 2.3979, 49855: 1.9924, 51468: 2.3979, 63600: 2.3979, 75300: 2.3979, 77757: 1.9924, 89318: 1.9924, 96799: 2.3979, 110980: 2.3979, 123553: 1.9924, 160081: 2.3979, 160086: 1.2993, 167255: 1.2993, 178931: 2.3979, 208192: 2.3979, 217323: 3.9849, 257249: 2.3979, 261393: 2.3979}))
Row(idf=SparseVector(262144, {29823: 2.3979, 32228: 2.3979, 36822: 2.3979, 39431: 1.7047, 41144: 2.3979, 49855: 1.9924, 61014: 2.3979, 61103: 2.3979, 64713: 2.3979, 72971: 2.3979, 82902: 1.9924, 84159: 1.9924, 118725: 4.7958, 153204: 2.3979, 167255: 1.2993, 178229: 2.3979, 185989: 2.3979, 206065: 2.3979, 217323: 1.9924, 235673: 2.3979, 237148: 2.3979, 248593: 2.3979, 249689: 2.3979, 251574: 1.9924, 254458: 2.3979}))
Row(idf=SparseVector(262144, {118171: 1.4816, 160086: 

## 6. TF-IDF 컬럼을 features로 구성, 출력

In [20]:
idfDf=idfDf.withColumnRenamed('idf', 'features')
idfDf.select('value','features').show()

+----------------------------------+--------------------+
|                             value|            features|
+----------------------------------+--------------------+
|  존경하는 국민 여러분, 경찰관 ...|(262144,[162,8073...|
| 국민의 안전을 위해 밤낮없이 애...|(262144,[5341,630...|
|오늘 홍조근정훈장을 받으신 중앙...|(262144,[29823,32...|
|          사랑하는 경찰관 여러분,
|(262144,[118171,1...|
|여러분의 헌신적 노력으로 우리의...|(262144,[11202,28...|
| 치안의 개선은 국민의 체감으로 ...|(262144,[883,9986...|
| 한국을 찾는 외국 관광객들도 우...|(262144,[7183,239...|
|   올해는 ‘경찰의 날’에 맞춰 국...|(262144,[554,4988...|
|        자랑스러운 경찰관 여러분,
|(262144,[118171,1...|
| 경찰헌장은 “나라와 겨레를 위하...|(262144,[4628,201...|
|    대한민국 경찰은 1945년 광복...|(262144,[32368,53...|
| 임시정부 초대 경무국장 백범 김...|(262144,[17472,23...|
|    광복 이후 6·25전쟁에서도 경...|(262144,[10556,12...|
|그러나 잘못도 없지는 않았습니다...|(262144,[20328,20...|
| 지금 경찰은 과거를 돌아보며 국...|(262144,[4505,125...|
| 검경 수사권 조정과 자치경찰제 ...|(262144,[13448,18...|
|정부는 경찰의 근무여건을 개선하...|(262144,[514,8701...|
| 정부는 누구도 법 위에 군림하지...|(262144,[6358,160...|
| 경찰헌장

In [21]:
## word2Vec 이용

In [22]:
from pyspark.ml.feature import Word2Vec

word2vec = Word2Vec(vectorSize = 20, minCount = 1, inputCol = "wordsReg", outputCol = "w2v")
model = word2vec.fit(reDf)
w2vDf = model.transform(reDf)

In [23]:
model.getVectors().show()

+-----------+--------------------+
|       word|              vector|
+-----------+--------------------+
|   공정해야|[-0.0146769583225...|
| 자치경찰제|[-0.0211083367466...|
|   공권력을|[-0.0164100863039...|
|흘렸습니다.|[-0.0077968100085...|
|     만들고|[0.00922949705272...|
|     헌신에|[-0.0098557695746...|
|    나창헌,|[-0.0041433367878...|
|       닿아|[0.01233200356364...|
|         돌|[0.01824185997247...|
|     민갑룡|[0.01333562098443...|
|     수상자|[0.00310536869801...|
|       기간|[0.00795323215425...|
|     시대적|[-0.0140350675210...|
|     일흔네|[0.00698785018175...|
|  했습니다.|[0.01522052101790...|
|       범죄|[-0.0131280673667...|
|  됐습니다.|[0.00331099424511...|
|       검경|[-0.0165771935135...|
|   공권력이|[-0.0179498121142...|
|     치안이|[-0.0085832616314...|
+-----------+--------------------+
only showing top 20 rows



In [24]:
model.findSynonyms("검경",5).show()

+---------+-------------------+
|     word|         similarity|
+---------+-------------------+
|   본연의| 0.5788632035255432|
|   사랑을| 0.4740690588951111|
|     없이|0.46875786781311035|
|열립니다.|0.46254974603652954|
|   박정훈| 0.4623321294784546|
+---------+-------------------+



In [25]:
model.findSynonyms("치안이",5).show()

+------+-------------------+
|  word|         similarity|
+------+-------------------+
|잘못도| 0.6951565146446228|
|충실히| 0.6312954425811768|
|목숨을| 0.5522195100784302|
|  가장| 0.5389007329940796|
|동시에|0.47095292806625366|
+------+-------------------+

